In [21]:
import pandas as pd
import numpy as np
import nltk
import json

In [22]:
%run constants.ipynb
%run functions.ipynb
%run dictionnaries.ipynb

In [23]:
%run tag_to_filter.v1.ipynb

In [24]:
df_transitions = pd.read_csv(data_directory+'df_transitions.csv', 
                             sep='§', 
                             engine='python',
                             index_col=0, 
                             encoding='utf-8')

In [25]:
class Markov:
    def __init__(self, name, transitions):
        self.name = name
        self.transitions = transitions
        
    def predict(self, event):
        predict = predict_next_state(event, self.transitions)
        return predict
    
    def random_state(self):
        return np.random.choice(self.transitions.index)
    
markov = Markov('Markov', df_transitions)
markov.predict(markov.random_state())

'{"filters": {"manufacturer": ["boeing"], "aircraft": ["b_52_stratofortress"], "company": [], "country": [], "category": ["business", "premium"], "date": ["10112013", "16062017"]}, "tab": "default"}'

In [26]:
class Intent:
    def __init__(self, name):
        self.name = name

class Afficheur(Intent):
    def __init__(self, name, model_tagger, model_predicteur):
        self.name = name
        self.tagger = model_tagger
        self.predicteur = model_predicteur
    
    def get_tags(self, sentence):
        #request = self.tagger.predict([sentence])[0]
        request = [{'in': 'O'},{'Sweden': 'COUN_B'},{'France': 'COUN_B'},
                   {'from': 'O'},{'SPRING': 'DATE1_B'},{'1902': 'DATE1_B'},
                   {'to': 'O'},{'AUTUMN': 'DATE2_B'},{'2017': 'DATE2_E'},
                   {'cloud': 'STAT_B'},{'of': 'STAT_I'},{'words': 'STAT_E'},
                   {'for': 'O'},{'Pilatus': 'MANU_B'},{'Pilatus': 'O'}]
        return request
    
    def get_filters(self, sentence):
        request = self.get_tags(sentence)
        tags_values = extract_tags(request)
        filters = tag_to_filters(tags_values)
        filters = apply_date(filters)
        event = {
            CT_tabs : CT_tabs_default,
            CT_filt : filters,
        }
        return event
    
    def interact(self, sentence):
        event = self.get_filters(sentence)
        event_state = json.dumps(event)
        event_state_hash = json_string_to_hash(event_state)
        pred_state_hash = self.predicteur.predict(event_state_hash)
        pred_sentence = make_sentence_fom_json(json.loads(pred_state_hash))
        
        res = {"in_sate" : event_state_hash,
               "out_state" : pred_state_hash,
               "out_sentence" : pred_sentence,}
        
        return res
    
Afficheur_Intent = Afficheur(name='Afficheur', model_tagger='tagger', model_predicteur=markov)
Afficheur_Intent.name

'Afficheur'

In [27]:
class Classifieur:
    def __init__(self, name, model_classifieur):
        self.name = name
        self.classifieur = model_classifieur
        
    def predict(self, sentence):
        return 0

Classifieur = Classifieur(name='Intent classifieur', model_classifieur="")
Classifieur.predict(sentence="")

0

In [28]:
intents  = {"classifieur" : Classifieur,
            "intents" : {0 : Afficheur_Intent}
           }

intents

{'classifieur': <__main__.Classifieur at 0x7f322665bc18>,
 'intents': {0: <__main__.Afficheur at 0x7f32266635f8>}}

In [19]:
class ChatBot:
    def __init__(self, name, intents={}, classifieur=""):
        self.name = name
        self.classifieur = intents['classifieur']
        self.intents = intents['intents']
        
    def get_intent_classification(self, sentence):
        return self.classifieur.predict(sentence)
    
    def interact(self, sentence):
        intent_id = self.get_intent_classification(sentence)
        intent = self.intents[intent_id]
        interaction = intent.interact(sentence)
        return interaction

    
    
Hubert = ChatBot(name='Chatbot Hubert', intents=intents)
Hubert.get_intent_classification('aertetreb') 

0

In [20]:
res = Hubert.interact('aertetreb')
for k,v in res.items():
    print(k,':\t',v,'\n')

in_sate :	 {"filters": {"manufacturer": ["pilatus"], "aircraft": [], "company": [], "country": ["france", "sweden"], "category": [], "date": ["01031902", "30112017"]}, "tab": "default"} 

out_state :	 {"filters": {"manufacturer": ["airbus", "cessna", "cirrus ", "matra"], "aircraft": ["11e_spartan_185", "430", "hunter", "s_2_tracker"], "company": ["tam_airlines"], "country": ["anguilla", "costa_rica", "libya", "togo"], "category": [], "date": ["06082017", "09022010"]}, "tab": "seat"} 

out_sentence :	 We suggest you theseats' study from 06-08-2017 to 09-02-2010 for the manufacturers airbus,cessna,cirrus ,matra and for the aircrafts 11e_spartan_185,430,hunter,s_2_tracker and for the companytam_airlines and. If you agree, click onto the following link ;) 

